In [1]:
import utils

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_similarity_score

import keras
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer
from keras.optimizers import RMSprop, Adam, SGD

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy

from os.path import join, basename, exists
from os import makedirs, listdir

/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [46]:
def preproc_target_train(data, reduce_classes=False):
    diagnoses = data['Код_диагноза'].copy()
    
    if reduce_classes:
        pop_diagnoses = set(utils.get_most_popular_diagnoses(diagnoses, percent=.80))
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    else:
        pop_diagnoses = set(diagnoses)
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def preproc_target_test(data, pop_diagnoses, most_pop_diagnose):
    diagnoses = data['Код_диагноза'].copy()
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def join_topics(data, topics):
    data = data.copy()
    
    topics_df = pd.DataFrame(dict(zip(
        ['topic' + str(i) for i in range(topics.shape[1])],
        [topics[:, i] for i in range(topics.shape[1])])))
    topics_df['Id_Записи'] = data['Id_Записи']

    data = data.join(topics_df, on='Id_Записи', rsuffix='_topics', how='outer')
    data = data.drop(columns=['Id_Записи_topics'])
    
    return data

In [106]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

class DoctorsPopularityTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        doctors = x.fillna('sss')
        doctors_voc, counts = np.unique(doctors, return_counts=True)
        self.pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
        
        return self

    def transform(self, x):
        x = x.fillna('sss')
        x[x == 'sss'] = self.pop_doctor
        
        return x
    
class GenderTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = x.copy()
        x[x == 1] = 0
        x[x == 2] = 1
        
        return np.expand_dims(x, axis=1)
    
class AgeTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.expand_dims(x, axis=1)
    
class TopicsTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return x

class ClinicTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        clinics, counts = np.unique(x, return_counts=True)
        self.clinics_to_idx = dict(zip(clinics, range(len(clinics))))
        
        self.most_pop_clinic_idx = np.argmax(counts)
        
        return self
    
    def transform(self, x):
        x_vec = np.zeros((x.shape[0], len(self.clinics_to_idx)), dtype=np.float32)
        for i, clinic in enumerate(x):
            if clinic in self.clinics_to_idx:
                x_vec[i, self.clinics_to_idx[clinic]] = 1.
            else:
                x_vec[i, self.most_pop_clinic_idx] = 1.
        
        return x_vec
    
class RepeatsTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return np.expand_dims(x, axis=1)

In [107]:
class NNModel(BaseEstimator, TransformerMixin):
    def __init__(self, batch_size, epochs, tb_log_dir, model_path, validation_split=0.3):
        self.batch_size = batch_size
        self.epochs = epochs
        self.tb_log_dir = tb_log_dir
        self.model_path = model_path
        self.validation_split = validation_split
    
    def fit(self, x, y=None):
        self.classes = np.unique(y)
        self.classes_voc = dict(zip(self.classes, range(self.classes.shape[0])))
        self.voc_classes = dict(zip(range(self.classes.shape[0]), self.classes))
        y_proc = np.zeros((y.shape[0], self.classes.shape[0]), dtype=np.float32)
        for i, yc in enumerate(y):
            y_proc[i, self.classes_voc[yc]] = 1.
        
        self.model = Sequential([
            InputLayer(input_shape=(x.shape[1],)),
            Dense(2048, activation='sigmoid'),
            Dense(2048, activation='sigmoid'),
#             Dense(4096, activation='sigmoid'),
#             Dense(4096, activation='sigmoid'),
            Dense(self.classes.shape[0], activation='softmax')
        ])
        
        optim = RMSprop(lr=1e-4, decay=1e-6)
        self.model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.model.fit(x, y_proc,
                  batch_size=self.batch_size, epochs=self.epochs,
                  callbacks=[
                      TensorBoard(log_dir=self.tb_log_dir, batch_size=self.batch_size),
                      ModelCheckpoint(filepath=self.model_path, monitor='acc', period=5)
                  ],
                  validation_split=self.validation_split)
        
        return self
        
    def transform(self, x):
        pred = self.model.predict(x)
        max_idxs = np.argmax(pred, axis=1)
        
        return np.array(list(map(lambda max_idx: self.voc_classes[max_idx], max_idxs)))

In [108]:
no_validation = False
validation_split = 0.3

experiment_dir = 'simple_models'
model_name = 'nn_3dense' + str(utils.get_next_model_id(experiment_dir))
tb_log_dir = join(experiment_dir, 'log', model_name)
models_dir = join(experiment_dir, 'models')

if no_validation:
    model_path = utils.get_model_fname_pattern(models_dir, model_name, no_validation=True)
    validation_split = 0.0
else:
    model_path = utils.get_model_fname_pattern(models_dir, model_name, no_validation=False)

pipe = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('complaints_pipe', Pipeline([
                ('complaint_selector', ItemSelector(key='Жалобы (unigramm)')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=10, stop_words=stopwords.words('russian')))
            ])),
            ('complaints_n_pipe', Pipeline([
                ('complaint_n_selector', ItemSelector(key='Жалобы (ngramm)')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=1, stop_words=stopwords.words('russian')))
            ])),
            ('doctor_pipe', Pipeline([
                ('doctor_selector', ItemSelector(key='Врач')),
                ('doc_pop', DoctorsPopularityTransformator()),
                ('count_vect', CountVectorizer())
            ])),
            ('gender_pipe', Pipeline([
                ('gender_selector', ItemSelector(key='Пол')),
                ('gender_transform', GenderTransformator())
            ])),
            ('age_pipe', Pipeline([
                ('age_selector', ItemSelector(key='Возраст')),
                ('age_transformator', AgeTransformator())
            ])),
            ('topics_pipe', Pipeline([
                ('topic_selector', ItemSelector(key=['topic' + str(i) for i in range(355)])),
                ('topics_transform', TopicsTransformator())
            ])),
            ('clinic_pipe', Pipeline([
                ('clinic_selector', ItemSelector(key='Клиника')),
                ('clinic_transform', ClinicTransformator())
            ])),
            ('repeats_pipe', Pipeline([
                ('repeats_selector', ItemSelector(key='Повторный приём')),
                ('repeats_transform', RepeatsTransformator())
            ]))
        ]
    )),
    ('clf', NNModel(batch_size=128, epochs=200, tb_log_dir=tb_log_dir,
                    model_path=model_path, validation_split=validation_split))
])

In [109]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
train_topics = np.load('data/topics_train_ngramm.npy')
train = join_topics(train, train_topics)
# train, valid = train_test_split(train, test_size=0.3)

In [110]:
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)
# valid_y, _, _ = preproc_target_test(valid, pop_diagnoses, most_pop_diagnose)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [111]:
np.unique(train_y).shape

(2302,)

In [ ]:
pipe.fit(train, train_y)

Train on 43383 samples, validate on 18593 samples
Epoch 1/200
43383/43383 [==============================] - 8s 180us/step - loss: 5.6875 - acc: 0.0505 - val_loss: 6.1644 - val_acc: 0.0463
Epoch 2/200
43383/43383 [==============================] - 8s 176us/step - loss: 5.4402 - acc: 0.0767 - val_loss: 6.0659 - val_acc: 0.0805
Epoch 3/200
43383/43383 [==============================] - 8s 173us/step - loss: 5.1680 - acc: 0.1007 - val_loss: 5.7823 - val_acc: 0.1002
Epoch 4/200
43383/43383 [==============================] - 8s 179us/step - loss: 4.8725 - acc: 0.1392 - val_loss: 5.5801 - val_acc: 0.1458
Epoch 5/200
43383/43383 [==============================] - 8s 177us/step - loss: 4.6273 - acc: 0.1709 - val_loss: 5.3343 - val_acc: 0.1569
Epoch 6/200
43383/43383 [==============================] - 8s 177us/step - loss: 4.3945 - acc: 0.1871 - val_loss: 5.1719 - val_acc: 0.1717
Epoch 7/200
43383/43383 [==============================] - 8s 182us/step - loss: 4.1879 - acc: 0.2124 - val_loss: 4.

43383/43383 [==============================] - 8s 174us/step - loss: 2.6905 - acc: 0.3574 - val_loss: 3.8587 - val_acc: 0.3050
Epoch 60/200
43383/43383 [==============================] - 8s 177us/step - loss: 2.6857 - acc: 0.3584 - val_loss: 3.8553 - val_acc: 0.3037
Epoch 61/200
43383/43383 [==============================] - 8s 177us/step - loss: 2.6794 - acc: 0.3586 - val_loss: 3.8560 - val_acc: 0.3047
Epoch 62/200
43383/43383 [==============================] - 8s 177us/step - loss: 2.6740 - acc: 0.3604 - val_loss: 3.8603 - val_acc: 0.3022
Epoch 63/200
43383/43383 [==============================] - 8s 176us/step - loss: 2.6688 - acc: 0.3608 - val_loss: 3.8422 - val_acc: 0.3060
Epoch 64/200
43383/43383 [==============================] - 8s 175us/step - loss: 2.6618 - acc: 0.3613 - val_loss: 3.8581 - val_acc: 0.3038
Epoch 65/200
43383/43383 [==============================] - 8s 175us/step - loss: 2.6559 - acc: 0.3627 - val_loss: 3.8319 - val_acc: 0.3042
Epoch 66/200
43383/43383 [=======

In [18]:
pred = pipe.transform(train)

In [19]:
np.unique(pred, return_counts=True)

(array(['A46', 'A63.0', 'B00', 'B00.1', 'B00.8', 'B02', 'B07', 'B08.1',
        'B35.1', 'B35.3', 'B35.6', 'B36.0', 'B37', 'B37.3+', 'B97.1',
        'B97.7', 'D17.0', 'D17.1', 'D17.2', 'D18.0', 'D23', 'D23.3',
        'D23.5', 'D23.9', 'D24', 'D25', 'D50', 'D50.9', 'D89.8', 'E01.1',
        'E01.8', 'E03.8', 'E03.9', 'E04.1', 'E04.2', 'E05.0', 'E06.3',
        'E11', 'E22.1', 'E28', 'E28.1', 'E28.8', 'E28.9', 'E66.0', 'E89.0',
        'F41.0', 'F41.2', 'F45.3', 'F95.0', 'G24', 'G24.9', 'G43.0',
        'G44.2', 'G50.0', 'G51.0', 'G55*', 'G55.1*', 'G55.3*', 'G56.0',
        'G56.2', 'G90', 'G90.8', 'G90.9', 'G93.4', 'H00', 'H00.0', 'H00.1',
        'H01.0', 'H04.1', 'H10.0', 'H10.1', 'H10.2', 'H10.3', 'H10.5',
        'H10.8', 'H11.3', 'H15.1', 'H16.1', 'H16.2', 'H20.0', 'H35.0',
        'H35.3', 'H35.4', 'H40.1', 'H52.0', 'H52.1', 'H52.2', 'H52.4',
        'H52.5', 'H60', 'H61.2', 'H65', 'H65.0', 'H65.1', 'H66.0', 'H66.1',
        'H68.0', 'H68.1', 'H81.1', 'H81.2', 'H90.3', 'H93.1', 

## Submit

In [9]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
train_topics = np.load('data/topics_train_ngramm.npy')
train = join_topics(train, train_topics)
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [10]:
pipe.fit(train, train_y)

Epoch 1/100
61976/61976 [==============================] - 13s 215us/step - loss: 5.8733 - acc: 0.0998
Epoch 2/100
61976/61976 [==============================] - 13s 212us/step - loss: 5.5176 - acc: 0.1808
Epoch 3/100
61976/61976 [==============================] - 13s 211us/step - loss: 5.1080 - acc: 0.2237
Epoch 4/100
61976/61976 [==============================] - 13s 212us/step - loss: 4.7338 - acc: 0.2444
Epoch 5/100
61976/61976 [==============================] - 13s 211us/step - loss: 4.4498 - acc: 0.2577
Epoch 6/100
61976/61976 [==============================] - 13s 211us/step - loss: 4.2481 - acc: 0.2679
Epoch 7/100
61976/61976 [==============================] - 13s 209us/step - loss: 4.1090 - acc: 0.2773
Epoch 8/100
61976/61976 [==============================] - 13s 209us/step - loss: 4.0043 - acc: 0.2860
Epoch 9/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.9136 - acc: 0.2952
Epoch 10/100
61976/61976 [==============================] - 13s 211us/ste

61976/61976 [==============================] - 13s 207us/step - loss: 3.1456 - acc: 0.3923
Epoch 80/100
61976/61976 [==============================] - 13s 208us/step - loss: 3.1393 - acc: 0.3923
Epoch 81/100
61976/61976 [==============================] - 13s 210us/step - loss: 3.1382 - acc: 0.3930
Epoch 82/100
61976/61976 [==============================] - 13s 207us/step - loss: 3.1367 - acc: 0.3942
Epoch 83/100
61976/61976 [==============================] - 13s 208us/step - loss: 3.1333 - acc: 0.3947
Epoch 84/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.1251 - acc: 0.3940
Epoch 85/100
61976/61976 [==============================] - 13s 207us/step - loss: 3.1240 - acc: 0.3963
Epoch 86/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.1194 - acc: 0.3956
Epoch 87/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.1168 - acc: 0.3966
Epoch 88/100
61976/61976 [==============================] - 13s 208us/step - 

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('complaints_pipe', Pipeline(memory=None,
     steps=[('complaint_selector', ItemSelector(key='Жалобы (unigramm)')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, e...:48_2018/{epoch:02d}_{acc:.2f}.h5',
    tb_log_dir='simple_models/log/nn14', validation_split=0.0))])

In [101]:
test = utils.load_data('data/test_data_complaints_repeats_doctors.csv')
test_topics = np.load('data/topics_test_ngramm.npy')
test = join_topics(test, test_topics)

In [102]:
test_pred = pipe.transform(test)

In [103]:
submit = pd.DataFrame({'Id_Записи': test['Id_Записи'], 'Код_диагноза': test_pred})

In [104]:
submit['Код_диагноза'].value_counts()

M42.1      2324
J06.9      1897
N76.0      1620
Z32.1      1499
J00        1002
N41.1       935
M51         901
I11         818
Z00.0       804
N77.1*      637
M65         564
K30         534
K29.9       527
J35.0       506
Z01.8       501
M54.8       468
N60.1       442
H52.1       409
Z01.4       392
N76.1       375
D25         364
K29.5       315
D23.9       314
I83.9       264
L30.8       262
M54.2       259
H61.2       252
M53.8       246
M54.4       231
G90         218
           ... 
I70.2         2
H68.1         2
M20-M25       2
H65.1         2
S20.2         2
K21.9         2
E14           2
E10           2
I25.0         2
M79.2         2
J35.3         2
M51.2         1
I87.0         1
H66.0         1
I67.4         1
I87.2         1
I83           1
N83.2         1
G44.1         1
E89.0         1
L30.3         1
L52           1
N43.0         1
L04.0         1
S90.1         1
K58.9         1
N48.8         1
M76.0         1
M72.5         1
H40.1         1
Name: Код_диагноза, Leng

In [105]:
submit.to_csv('submit/bow_simple_nn32_3dense_diag_all_compl_uni-n_gram_doctor_topics_clinic.csv', header=True, index=False)